In [1]:
"""
Exercise 02: Sub-Agent Context - Starter Code
==============================================
Extend supervisor with context engineering.

LEARNING GOALS:
- Use ToolRuntime to access conversation context
- Use InjectedToolCallId for proper ToolMessage returns
- Use Command for state updates
"""

import os
from typing import Annotated
from dotenv import load_dotenv

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain.tools import ToolRuntime
from langchain_core.tools import InjectedToolCallId
from langchain_core.messages import ToolMessage
from langgraph.types import Command

load_dotenv()

model = init_chat_model("gpt-4o-mini", model_provider="openai")


/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/Recature-Homework/Revature-Cognizant-Training-Exercises/January/venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
# =============================================================================
# TODO 2: Create Sub-Agents
# =============================================================================
# Create research_agent and writing_agent similar to Exercise 1.
# These will be invoked by your context-aware tools.
# =============================================================================

research_agent = create_agent(
    name="research_agent",
    model=model,
    tools=[],
    system_prompt="""You are a research specialist. Your role is to gather comprehensive information about topics.
    
CRITICAL: You must include ALL findings in your response. Sub-agents only return their final message, so ensure your response contains complete research results."""
)

writing_agent = create_agent(
    name="writing_agent",
    model=model,
    tools=[],
    system_prompt="""You are a writing specialist. Your role is to create polished, well-structured content based on research provided to you.
    
CRITICAL: You must include COMPLETE content in your response. Sub-agents only return their final message, so ensure your response contains the full written content."""
)

In [3]:
# =============================================================================
# TODO 3: Implement Research with Context
# =============================================================================
# Create a tool that:
# 1. Takes topic and runtime: ToolRuntime parameters
# 2. Gets conversation history from runtime.state.get("messages", [])
# 3. Formats recent context for the sub-agent
# 4. Invokes research_agent with the context
# 5. Returns the research result
#
# EXPERIMENT: How much context is helpful? Too much?
# EXPERIMENT: Does summarizing history vs raw messages matter?
# =============================================================================

@tool
def research_with_context(topic: str, runtime: ToolRuntime) -> str:
    """Research with access to conversation context."""
    messages = runtime.state.get("messages", [])
    
    recent_context = "\n".join([
        f"{getattr(msg, 'type', 'unknown')}: {getattr(msg, 'content', str(msg))}" 
        for msg in messages[-5:] if hasattr(msg, 'content') or isinstance(msg, dict)
    ])
    
    context_prompt = f"Research the following topic: {topic}"
    if recent_context:
        context_prompt += f"\n\nRecent conversation context:\n{recent_context}"
    
    result = research_agent.invoke({
        "messages": [{"role": "user", "content": context_prompt}]
    })
    return result["messages"][-1].content

In [4]:
# =============================================================================
# TODO 4: Implement Writing with State Tracking
# =============================================================================
# Create a tool that:
# 1. Takes topic, research, tool_call_id (InjectedToolCallId), runtime
# 2. Invokes writing_agent
# 3. Gets existing topics_covered from runtime.state
# 4. Returns Command with:
#    - update containing ToolMessage and updated topics_covered list
#
# The Command allows you to both return a tool result AND update state!
#
# EXPERIMENT: What else could you track? Word count? Sentiment?
# =============================================================================

@tool
def write_with_tracking(
    topic: str, 
    research: str,
    tool_call_id: Annotated[str, InjectedToolCallId],
    runtime: ToolRuntime
) -> Command:
    """Write content and track topics covered."""
    result = writing_agent.invoke({
        "messages": [{"role": "user", "content": f"Write polished content about: {topic}\n\nResearch findings:\n{research}"}]
    })
    
    content = result["messages"][-1].content
    
    topics = runtime.state.get("topics_covered", [])
    
    return Command(update={
        "messages": [ToolMessage(content=content, tool_call_id=tool_call_id)],
        "topics_covered": topics + [topic] if topic not in topics else topics
    })

In [5]:
# =============================================================================
# TODO 5: Create Supervisor Agent
# =============================================================================
# Create a supervisor that uses your context-aware tools.
# Include in prompt that it should track topics covered.
# =============================================================================

from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=[research_with_context, write_with_tracking],
    checkpointer=InMemorySaver(),
    system_prompt="""You are a supervisor agent coordinating research and writing tasks with context awareness.

Your workflow should be:
1. Use research_with_context to gather information about topics, leveraging conversation history
2. Use write_with_tracking to create polished content based on research
3. Track topics covered throughout the conversation
4. Present the final written content to the user

You have access to conversation context, so use it to make your research more relevant and contextual.""",
    name="supervisor_agent"
)

In [6]:
# =============================================================================
# Testing the Supervisor Agent with Context
# =============================================================================

config = {"configurable": {"thread_id": "test_session"}}

print("Test with multiple related requests:")
print("=" * 80)
print()

test_prompt_1 = "Write about Python basics"
print(f"1. User: {test_prompt_1}")
print("-" * 80)
result1 = agent.invoke({
    "messages": [{"role": "user", "content": test_prompt_1}]
}, config)
print(f"Agent: {result1['messages'][-1].content[:200]}...")
print()

test_prompt_2 = "Now write about advanced Python"
print(f"2. User: {test_prompt_2}")
print("-" * 80)
result2 = agent.invoke({
    "messages": [{"role": "user", "content": test_prompt_2}]
}, config)
print(f"Agent: {result2['messages'][-1].content[:200]}...")
print()

test_prompt_3 = "What topics have we covered?"
print(f"3. User: {test_prompt_3}")
print("-" * 80)
result3 = agent.invoke({
    "messages": [{"role": "user", "content": test_prompt_3}]
}, config)
print(f"Agent: {result3['messages'][-1].content}")
print()

print("=" * 80)
print("\nCheck LangSmith to see the tool call sequence and context usage!")

Test with multiple related requests:

1. User: Write about Python basics
--------------------------------------------------------------------------------
Agent: Here is a comprehensive overview of Python basics:

# Python Basics: A Comprehensive Guide

## 1. Introduction to Python
Python is a high-level, interpreted programming language renowned for its reada...

2. User: Now write about advanced Python
--------------------------------------------------------------------------------
Agent: Here’s a comprehensive overview of advanced Python concepts:

# Advanced Python: A Comprehensive Guide

Advanced Python programming delves into the intricacies of Python's capabilities, focusing on as...

3. User: What topics have we covered?
--------------------------------------------------------------------------------
Agent: We have covered the following topics in this conversation:

1. **Python Basics:**
   - Introduction to Python
   - Installation
   - Basic Syntax
   - Data Types
   - Control